In [19]:
import numpy as np
import pandas as pd

In [20]:
model_name_split = 'gpt2-xl'
model_name_split = 'gpt-j-6b'
model_name_split = 'Meta-Llama-3.1-8B'

dataset = 'mquake'
if 'gpt-j' in model_name_split:
    seeds = [i+1 for i in range(5)]
elif 'gpt2-xl' in model_name_split:
    seeds = [i+1 for i in range(5)]
elif 'llama' in model_name_split.lower():
    seeds = [i+1 for i in range(5)]

In [ ]:
dfs = []
for s in seeds: 
    df = pd.read_csv(f'../src/context-dims/{dataset}/{s}/results_tuned.csv', names = ['model', 'setting', 'entropy', 'epochs', '#reversal_tokens', 'repeat_factor', 'lweight', 'det. acc.', 'match acc.'])
    df['seed'] = [s]*len(df)
    dfs.append(df)
final_df = pd.concat(dfs)
final_df.head(2)

In [ ]:
final_df = final_df[final_df['setting'] == 'unedited vs. tuned']
final_df = final_df[final_df['model'].str.contains(model_name_split)]
len(final_df)


In [ ]:
median_value = final_df['match acc.'].median()
final_df = final_df[final_df.apply(lambda x: np.isclose(x['match acc.'], median_value), axis=1)]
print(len(final_df))

seeds = final_df['seed'].values
print(final_df['match acc.'].median())
seeds

In [ ]:
final_df['match acc.']

In [ ]:
init_e = [np.loadtxt(f'../src/context-dims/{dataset}/{seed}/{model_name_split}_epochs_3_rt_1_rf_1_[reversal_token_1]_lambda_0.5_init.txt') for seed in seeds ]
final_e = [np.loadtxt(f'../src/context-dims/{dataset}/{seed}/{model_name_split}_epochs_3_rt_1_rf_1_[reversal_token_1]_lambda_0.5.txt') for seed in seeds ]
print(len(init_e))
assert len(init_e) == len(final_e)

In [ ]:
init_e

In [27]:
init_e = np.mean(init_e, axis=0)
final_e = np.mean(final_e, axis=0)

In [ ]:
init_e

In [29]:
weights = np.abs(init_e - final_e) /np.sum(np.abs(init_e - final_e))

In [ ]:
np.median(weights)*100

In [ ]:
weights.max()*100

In [ ]:
weights.min()*100

In [ ]:
weights_inverted = 1/weights
weights_inverted = weights_inverted/weights_inverted.sum()
weights_inverted.sum()

In [ ]:
weights_inverted.max()

In [ ]:
near_zero_count = np.sum( weights > 0.001)

print(f"Number of elements near zero: {near_zero_count}")

In [ ]:
np.savetxt(f'../context-weights/{dataset}/{model_name_split}.txt', weights) # which dimensions are important for resetting context     

In [37]:
np.savetxt(f'../context-weights/{dataset}/{model_name_split}_inverted.txt', weights_inverted) # which dimensions should be given higher weights in cosloss (penalize changes across these dimensions)